<a href="https://colab.research.google.com/github/juumplerjw/OSS_01/blob/main/WEEK09_OSS_%EA%B9%80%EC%A0%95%EC%9A%B0_20232018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

target()

running inner()


In [3]:
target

<function __main__.deco.<locals>.inner()>

In [7]:
def logger(func):
    def wrapper(*args, **kwargs):
        print(f"[LOG] {func.__name__} called with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"[LOG] {func.__name__} returned {result}")
        return result
    return wrapper

@logger
def add(a, b):
    return a + b

add(5, 12)

[LOG] add called with args=(5, 12), kwargs={}
[LOG] add returned 17


17

In [8]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x7b327bdcc860>)
running register(<function f2 at 0x7b3264937420>)
running main()
registry -> [<function f1 at 0x7b327bdcc860>, <function f2 at 0x7b3264937420>]
running f1()
running f2()
running f3()


In [9]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

In [10]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [11]:
def f1(a):
    print(a)
    print(b)

b=6
f1(3)

3
6


In [12]:
b = 6
def f2(a):
    print(a)
    print(b)
    b=9

f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [13]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [14]:
b

9

In [15]:
f3(3)

3
9


In [16]:
b=30
f3(3)

3
30


In [18]:
class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)


In [19]:
avg = Averager()
avg(10)

10.0

In [20]:
avg(11)

10.5

In [21]:
avg(12)

11.0

In [22]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager


In [23]:
avg = make_averager()
avg(10)

10.0

In [24]:
avg(11)

10.5

In [25]:
avg(12)

11.0

In [26]:
avg.__code__.co_varnames

('new_value', 'total')

In [27]:
avg.__closure__

(<cell at 0x7b3265018f40: list object at 0x7b32637087c0>,)

In [28]:
avg.__code__.co_freevars

('series',)

In [29]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [30]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager


In [31]:
avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [32]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager


In [33]:
avg = make_averager()
avg(10)

10.0

In [34]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [35]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12315388s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000059s] factorial(1) -> 1
[0.00001268s] factorial(2) -> 2
[0.00001952s] factorial(3) -> 6
[0.00002496s] factorial(4) -> 24
[0.00003050s] factorial(5) -> 120
[0.00003825s] factorial(6) -> 720
6! = 720


In [36]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        arg_lst = []
        name = func.__name__
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [37]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12318993s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000072s] factorial(1) -> 1
[0.00001454s] factorial(2) -> 2
[0.00002098s] factorial(3) -> 6
[0.00002718s] factorial(4) -> 24
[0.00003314s] factorial(5) -> 120
[0.00004196s] factorial(6) -> 720
6! = 720


In [38]:
factorial.__name__

'factorial'

In [42]:
!git clone https://github.com/juumplerjw/OSS_01.git

Cloning into 'OSS_01'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 22.99 KiB | 2.55 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [43]:
!git add
!git commit -m "update"
!git push origin main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
